In [18]:
!pip install tensorflow


     |███████████████████████▏        | 328.6 MB 77.2 MB/s eta 0:00:02IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

     |████████████████████████████████| 454.3 MB 1.2 kB/s 
     |████████████████████████████████| 4.4 MB 44.5 MB/s 
     |████████████████████████████████| 129 kB 84.7 MB/s 
     |████████████████████████████████| 14.9 MB 98.5 MB/s 
     |████████████████████████████████| 57 kB 12.9 MB/s 
     |████████████████████████████████| 42 kB 3.6 MB/s 
     |████████████████████████████████| 1.2 MB 91.1 MB/s 
     |████████████████████████████████| 462 kB 82.4 MB/s 
     |████████████████████████████████| 65 kB 11.6 MB/s 
     |████████████████████████████████| 10.6 MB 63.2 MB/s 
     |█████████████████

In [26]:
!pip install opencv-python==4.5.1.48

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [27]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator ## NN 
import numpy as np
import matplotlib.pyplot as plt

import os
import pandas as pd
import glob
from PIL import Image

In [29]:
main_dir = "/work/Data"
train_dir = os.path.join(main_dir,"/work/Data/train_data")
# test_dir = os.path.join(main_dir,"test_images")
train_data_image = !ls /datasets/trainimage


# The clinicians have rated each image for the severity of diabetic retinopathy on a scale of 0 to 4. It is a multi-class problem with 5 target classes
0 - No DR, 1 - Mild, 2 - Moderate 3 - Severe, 4 - Proliferative DR

In [ ]:
Train_data = pd.read_csv("../input/aptos2019-blindness-detection/train.csv")
Test_data = pd.read_csv("../input/aptos2019-blindness-detection/test.csv")
train_0 = np.count_nonzero(Train_data["diagnosis"] == 0)
train_1 = np.count_nonzero(Train_data["diagnosis"] == 1)
train_2 = np.count_nonzero(Train_data["diagnosis"] == 2)
train_3 = np.count_nonzero(Train_data["diagnosis"] == 3)
train_4 = np.count_nonzero(Train_data["diagnosis"] == 4)

In [ ]:
# data split
from sklearn.model_selection import train_test_split
# load image and convert to and from NumPy array
from PIL import Image
from numpy import argmax
import numpy as np
import cv2
from PIL import Image
train_x = Train_data["id_code"]
train_labels = Train_data["diagnosis"]

train_pic_path= [os.path.join(train_dir, filename)for filename in train_x]
train_image = []
for i, img_path in enumerate(train_pic_path):
    img_path = img_path + ".png"
    train_image.append(img_path)
print(len(train_image))

data_out = [0,1,2,3,4]
label_int =  dict((c, i) for i, c in enumerate(data_out))
int_labe = dict((i, c) for i, c in enumerate(data_out))
interger_encoded = [label_int[char] for char in train_labels]

onehot_encoded = []
for value in interger_encoded:
    labe_n = [0 for _ in range(len(data_out))]
    labe_n[value] = 1
    onehot_encoded.append(labe_n)
train_onehot_labe = onehot_encoded

data_frame_work = pd.DataFrame({"Train_label": train_onehot_labe,
                               "Train_image": train_image,
                               "Triain_id": train_x},
                              columns=["Train_label","Train_image","Triain_id"])


Valida_dataset = data_frame_work.sample(n=500)
Train_dataset = data_frame_work.drop(Valida_dataset.index)

 # Image Analysis

In [ ]:

def image_analysis(dataframe, path):
    width_range = []
    height_range = []
    for i in range(dataframe.shape[0]):
        img = cv2.imread(path+dataframe.iloc[i]['id_code']+'.png')
        height, width, _ = img.shape
        width_range.append(width)
        height_range.append(height)
    return width_range, height_range


In [ ]:
width_range, height_range = image_analysis(Train_data, "../input/aptos2019-blindness-detection/train_images/")

In [ ]:
# check image ratio of width and height
import seaborn as sns
ratio_list = []

for i,j in zip(width_range,height_range):
    ratio = i/j
    ratio_list.append(ratio)

sns.distplot(ratio_list, hist=True, kde=True, 
             bins=int(180/5), color = 'darkblue', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})

In [ ]:
hh = max(height_range)
ww = max(width_range)
cc = 3
color = (0,0,0) # add 0 which is the same with backgroup
result = np.full((hh,ww,cc), color, dtype=np.uint8)
new_image = []
def pad_max(image_group): 
    '''
    image_group: list of image path
    '''
    for image in image_group:
        imge = cv2.imread(image)
        ht, wd, cc = imge.shape
        
        # compute center offset
        xx = (ww - wd) // 2
        yy = (hh - ht) // 2
        
        # copy img image into center of result image
        result[yy:yy+ht, xx:xx+wd] = imge
        
        new_image.append(imge)

In [8]:
train_data = !ls /datasets/trainimage 


In [9]:
# width, height check
def image_analysis(dataframe, path): 
    width_range = []
    height_range = []
    for i in range(dataframe.shape[0]):
        img = cv2.imread(path+dataframe.iloc[i]['id_code']+'.png')
        height, width, _ = img.shape
        width_range.append(width)
        height_range.append(height)
    return width_range, height_range

def wid_heigh(image_group):
    width_range = []
    height_range = []
    for i in image_group:
        height, width, _ = i.shape
        width_range.append(width)
        height_range.append(height)
    return width_range, height_range 

# Check all png have the same size
def ckeckList(lst):
  
    ele = lst[0]
    chk = True
      
    # Comparing each element with first item 
    for item in lst:
        if ele != item:
            chk = False
            break;
              
    if (chk == True): print("Equal")
    else: print("Not equal")   
        
def pad_max(image_group): 
    '''
    image_group: list of image path
    '''
    for image in image_group:
        imge = cv2.imread(image)
        ht, wd, cc = imge.shape
        
        # compute center offset
        xx = (ww - wd) // 2
        yy = (hh - ht) // 2
        
        # copy img image into center of result image
        result[yy:yy+ht, xx:xx+wd] = imge
        
        new_image.append(result)
        

3663

In [ ]:
width_range, height_range = image_analysis(Train_data, "../input/aptos2019-blindness-detection/train_images/")
# check image ratio of width and height
import seaborn as sns
ratio_list = []

for i,j in zip(width_range,height_range):
    ratio = i/j
    ratio_list.append(ratio)

sns.distplot(ratio_list, hist=True, kde=True, 
             bins=int(180/5), color = 'darkblue', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})
    
# add 0 to images to make it same size
hh = max(height_range)
ww = max(width_range)
cc = 3
color = (0,0,0) # add 0 which is the same with backgroup
result = np.full((hh,ww,cc), color, dtype=np.uint8)
new_image = []
  
# check results 
image_list = Train_dataset["Train_image"][1:200]
pad_max(image_list)
width_list, height_list  = wid_heigh(new_image)
lst1 = height_list
ckeckList(lst1)
lst2 = width_list
ckeckList(lst2)


# Model Part

In [ ]:
import tensorflow as tf

def FCN_model(len_classes=5, dropout_rate=0.2):
    
    input = tf.keras.layers.Input(shape=(None, None, 3))

    x = tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=1)(input)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    # x = tf.keras.layers.MaxPooling2D()(x)

    x = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    # x = tf.keras.layers.MaxPooling2D()(x)

    x = tf.keras.layers.Conv2D(filters=128, kernel_size=3, strides=2)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    # x = tf.keras.layers.MaxPooling2D()(x)

    x = tf.keras.layers.Conv2D(filters=256, kernel_size=3, strides=2)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    # x = tf.keras.layers.MaxPooling2D()(x)

    x = tf.keras.layers.Conv2D(filters=512, kernel_size=3, strides=2)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    # Uncomment the below line if you're using dense layers
    # x = tf.keras.layers.GlobalMaxPooling2D()(x)

    # Fully connected layer 1
    # x = tf.keras.layers.Dropout(dropout_rate)(x)
    # x = tf.keras.layers.BatchNormalization()(x)
    # x = tf.keras.layers.Dense(units=64)(x)
    # x = tf.keras.layers.Activation('relu')(x)

    # Fully connected layer 1
    x = tf.keras.layers.Conv2D(filters=64, kernel_size=1, strides=1)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    # Fully connected layer 2
    # x = tf.keras.layers.Dropout(dropout_rate)(x)
    # x = tf.keras.layers.BatchNormalization()(x)
    # x = tf.keras.layers.Dense(units=len_classes)(x)
    # predictions = tf.keras.layers.Activation('softmax')(x)

    # Fully connected layer 2
    x = tf.keras.layers.Conv2D(filters=len_classes, kernel_size=1, strides=1)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.GlobalMaxPooling2D()(x)
    predictions = tf.keras.layers.Activation('softmax')(x)

    model = tf.keras.Model(inputs=input, outputs=predictions)
    
    print(model.summary())
    print(f'Total number of layers: {len(model.layers)}')

    return model

# Generator Part

In [ ]:
class Generator(tf.keras.utils.Sequence):

    def __init__(self, DATASET_PATH, BATCH_SIZE=32, shuffle_images=True, image_min_side=24):
        """ Initialize Generator object.
        Args 
            DATASET_PATH           : Path to data dataframe
            BATCH_SIZE             : The size of the batches to generate.
            shuffle_images         : If True, shuffles the images read from the DATASET_PATH
            image_min_side         : After resizing the minimum side of an image is equal to image_min_side.
        """

        self.batch_size = BATCH_SIZE
        self.shuffle_images = shuffle_images
        self.image_min_side = image_min_side
        self.load_image_paths_labels(DATASET_PATH)
        self.create_image_groups()
   

    def load_image_paths_labels(self, DATASET_PATH):
        
        self.image_paths = DATASET_PATH["Train_image"]
        self.image_labels = DATASET_PATH["Train_label"]

        data_out = [0,1,2,3,4]
        label_int =  dict((c, i) for i, c in enumerate(data_out))
        int_labe = dict((i, c) for i, c in enumerate(data_out))
        interger_encoded = [label_int[char] for char in self.image_labels]

        onehot_encoded = []
        for value in interger_encoded:
            labe_n = [0 for _ in range(len(data_out))]
            labe_n[value] = 1
            onehot_encoded.append(labe_n)
        train_onehot_labe = onehot_encoded
    
        self.image_labels =  train_onehot_labe 
        
        assert len(self.image_paths) == len(self.image_labels)
        
    def create_image_groups(self):
        self.image_paths = np.asarray(self.image_paths)
        self.image_labels = np.asarray(self.image_labels)
        if self.shuffle_images:
            # Randomly shuffle dataset
            seed = 4321
            np.random.seed(seed)
            np.random.shuffle(self.image_paths)
            np.random.seed(seed)
            np.random.shuffle(self.image_labels)


        # Divide image_paths and image_labels into groups of BATCH_SIZE
        self.image_groups = [[self.image_paths[x % len(self.image_paths)] for x in range(i, i + self.batch_size)]
                              for i in range(0, len(self.image_paths), self.batch_size)]
        self.label_groups = [[self.image_labels[x % len(self.image_labels)] for x in range(i, i + self.batch_size)]
                              for i in range(0, len(self.image_labels), self.batch_size)]
        
    def resize_image(self, img, min_side_len):

        h, w, c = img.shape

        # limit the min side maintaining the aspect ratio
        if min(h, w) < min_side_len:
            im_scale = float(min_side_len) / h if h < w else float(min_side_len) / w
        else:
            im_scale = 1.

        new_h = int(h * im_scale)
        new_w = int(w * im_scale)

        re_im = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)
        return re_im, new_h / h, new_w / w
  

    def load_images(self, image_group):

        images = []
        for image_path in image_group:
            img = cv2.imread(image_path)
            img_shape = len(img.shape)
            if img_shape == 3:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img, rh, rw = self.resize_image(img, self.image_min_side)
            images.append(img)
        return images
  

    def construct_image_batch(self, image_group):
        # get the max image shape
        max_shape = tuple(max(image.shape[x] for image in image_group) for x in range(3))

        # construct an image batch object
        image_batch = np.zeros((self.batch_size,) + max_shape, dtype='float32')

        # copy all images to the upper left part of the image batch object
        for image_index, image in enumerate(image_group):
            image_batch[image_index, :image.shape[0], :image.shape[1], :image.shape[2]] = image

        return image_batch
   

    def __len__(self):
        """
        Number of batches for generator.
        """

        return len(self.image_groups)
 

    def __getitem__(self, index):
        """
        Keras sequence method for generating batches.
        """
        image_group = self.image_groups[index]
        label_group = self.label_groups[index]
        images = self.load_images(image_group)
        image_batch = self.construct_image_batch(images)

        return np.array(image_batch), np.array(label_group)

In [10]:
def train(model, train_generator, val_generator, epochs = 50):
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

  
    history = model.fit_generator(generator=train_generator,
                                    steps_per_epoch=len(train_generator),
                                    epochs=epochs,
                                    validation_data=val_generator,
                                    validation_steps=len(val_generator))

    return history

In [ ]:
def run():
    
    if __name__ == "__main__":
    
        # Create FCN model
        model = FCN_model(len_classes=5, dropout_rate=0.2)

        # The below folders are created using utils.py
        train_dir = train_data
        val_dir = Valida_dataset
    
        # If you get out of memory error try reducing the batch size
        BATCH_SIZE=8
        train_generator = Generator(train_dir, BATCH_SIZE, shuffle_images=True, image_min_side=24)
        val_generator = Generator(val_dir, BATCH_SIZE, shuffle_images=True, image_min_side=24)

        EPOCHS=50
        history = train(model, train_generator, val_generator, epochs=EPOCHS)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e3b8005b-8f5d-4e6f-98ca-740c88479b4d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>